In [9]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pyreadstat as st
import numpy as np
import matplotlib.pyplot as plt


path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 1T\[MT09] Econometría de Datos de Panel\Clases prácticas\PS 1-20250507\data\greene97.dta"

df, meta = st.read_dta(path)
df.head(1)

,id,year,c,q,pf,lf
0,1,1970,1140640,0.952757,106650,0.534487


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      90 non-null     int64  
 1   year    90 non-null     int64  
 2   c       90 non-null     int64  
 3   q       90 non-null     float64
 4   pf      90 non-null     int64  
 5   lf      90 non-null     float64
dtypes: float64(2), int64(4)
memory usage: 4.3 KB


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

df['lnc'] = np.log(df['c'])
df['lnq'] = np.log(df['q'])
df['lnpf'] = np.log(df['pf'])


# Usamos patsy y statsmodels con fórmula
model = smf.ols('lnc ~ lnq + lf + lnpf + C(id)', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    lnc   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     3936.
Date:                Sun, 25 May 2025   Prob (F-statistic):          1.51e-101
Time:                        16:46:17   Log-Likelihood:                 130.09
No. Observations:                  90   AIC:                            -242.2
Df Residuals:                      81   BIC:                            -219.7
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      9.7059      0.193     50.258      0.0

In [10]:
# La constante es la interpretación directa para la firma 1. 

In [11]:
df['e'] = model.resid
residual_vectors = {}
a1, a2 = 0, 15

for i in range(1, 7):
    key = f"e{i}"
    residual_vectors[key] = df['e'].iloc[a1:a2].to_numpy()
    a1 += 15
    a2 += 15


residual_vectors

{'e1': array([ 0.02432919,  0.03943107,  0.0361558 ,  0.06326764,  0.03383255,
        -0.03527942,  0.00501393, -0.00227683, -0.04424655, -0.07449206,
        -0.02131798, -0.03653725, -0.03434179,  0.00372271,  0.042739  ]),
 'e2': array([-0.10923203, -0.06401408, -0.05154175,  0.04937864, -0.02657994,
        -0.03834923, -0.04274458, -0.02869517,  0.00099328, -0.02405765,
        -0.03628389,  0.0167852 ,  0.07643585,  0.11203921,  0.16586613]),
 'e3': array([-0.01960544,  0.04021187,  0.06468929,  0.06826004,  0.00621729,
        -0.01833514, -0.06534268, -0.06725798, -0.01970372, -0.05083023,
        -0.03362109,  0.02679329, -0.01241617,  0.03486742,  0.04607325]),
 'e4': array([-0.15604118, -0.1124052 , -0.03959208, -0.00645815, -0.05179785,
        -0.03847341, -0.01640768,  0.01686399,  0.09115261,  0.01158102,
         0.00304268,  0.10066229,  0.15746305,  0.02248174,  0.01792816]),
 'e5': array([ 0.02541318,  0.06836765,  0.09636187,  0.06912193, -0.03227485,
        -0.01

In [12]:
sigma = {}

for i in range(1, 7):
    e = residual_vectors[f"e{i}"].reshape(-1, 1)  # columna
    sigma_matrix = (e.T @ e) / 15
    sigma[f"sigma{i}"] = sigma_matrix[0, 0]  # escalar

for i in range(1, 7):
    print(f"sigma{i} =", sigma[f"sigma{i}"])

sigma1 = 0.0014794964812895254
sigma2 = 0.004935781935074677
sigma3 = 0.0018884325726703035
sigma4 = 0.005834425258586821
sigma5 = 0.002338333594424573
sigma6 = 0.0030316786223437637


In [14]:
df['w'] = np.nan

# Asignar pesos en bloques de 15 observaciones
for i in range(6):
    start = i * 15
    end = start + 15
    df.loc[start:end-1, 'w'] = sigma[f'sigma{i+1}']

# Asegurarse de que 'id' es categórica
df['id'] = df['id'].astype('category')

# Crear dummies de 'id', excluyendo una categoría para evitar multicolinealidad
dummies = pd.get_dummies(df['id'], prefix='id', drop_first=True)

# Crear la matriz X
X = pd.concat([df[['lnq', 'lf', 'lnpf']], dummies], axis=1)
X = sm.add_constant(X)

# Asegurar que X es numérica
X = X.astype(float)

# Variable dependiente
y = df['lnc'].astype(float)

# Pesos
weights = (1 / df['w']).astype(float)

# FGLS usando WLS (Weighted Least Squares)
fgls_model = sm.WLS(y, X, weights=weights).fit()
print(fgls_model.summary())

                            WLS Regression Results                            
Dep. Variable:                    lnc   R-squared:                       0.998
Model:                            WLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     5527.
Date:                Sun, 25 May 2025   Prob (F-statistic):          1.66e-107
Time:                        17:11:26   Log-Likelihood:                 138.46
No. Observations:                  90   AIC:                            -258.9
Df Residuals:                      81   BIC:                            -236.4
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.9423      0.162     61.263      0.0

In [ ]:
# Pythonista.

import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# --- Paso 1: Transformaciones logarítmicas ---
df['lnc'] = np.log(df['c'])
df['lnq'] = np.log(df['q'])
df['lnpf'] = np.log(df['pf'])

# --- Paso 2: OLS con fórmula estilo R ---
model = smf.ols('lnc ~ lnq + lf + lnpf + C(id)', data=df).fit()
df['e'] = model.resid

# --- Paso 3: Calcular sigma_i como varianza de los residuos por bloque de 15 ---
block_size = 15
n_blocks = df.shape[0] // block_size
sigmas = (
    df['e']
    .groupby(df.index // block_size)
    .apply(lambda x: np.mean(x**2))
    .to_dict()
)

# --- Paso 4: Asignar pesos ---
df['w'] = df.index // block_size
df['w'] = df['w'].map(sigmas)
df['w'] = 1 / df['w']  # pesos inversos a la varianza

# --- Paso 5: Crear matriz de diseño y ajustar WLS ---
# Usamos fórmula nuevamente, ya que `WLS.from_formula()` lo permite
fgls_model = smf.wls('lnc ~ lnq + lf + lnpf + C(id)', data=df, weights=df['w']).fit()
print(fgls_model.summary())


                            WLS Regression Results                            
Dep. Variable:                    lnc   R-squared:                       0.998
Model:                            WLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     5527.
Date:                Sun, 25 May 2025   Prob (F-statistic):          1.66e-107
Time:                        17:14:17   Log-Likelihood:                 138.46
No. Observations:                  90   AIC:                            -258.9
Df Residuals:                      81   BIC:                            -236.4
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      9.9423      0.162     61.263      0.0

In [16]:
def run_fgls(df, formula, group_size=15):
    df = df.copy()
    model = smf.ols(formula, data=df).fit()
    df['resid'] = model.resid

    sigmas = (
        df['resid']
        .groupby(df.index // group_size)
        .apply(lambda x: np.mean(x**2))
        .to_dict()
    )

    df['w'] = df.index // group_size
    df['w'] = df['w'].map(sigmas)
    df['w'] = 1 / df['w']

    fgls_model = smf.wls(formula, data=df, weights=df['w']).fit()
    return fgls_model


model = run_fgls(df, 'lnc ~ lnq + lf + lnpf + C(id)')
print(model.summary())

                            WLS Regression Results                            
Dep. Variable:                    lnc   R-squared:                       0.998
Model:                            WLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     5527.
Date:                Sun, 25 May 2025   Prob (F-statistic):          1.66e-107
Time:                        17:15:23   Log-Likelihood:                 138.46
No. Observations:                  90   AIC:                            -258.9
Df Residuals:                      81   BIC:                            -236.4
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      9.9423      0.162     61.263      0.0

In [17]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# 1. Modelo inicial OLS
df['lnc'] = np.log(df['c'])
df['lnq'] = np.log(df['q'])
df['lnpf'] = np.log(df['pf'])

model = smf.ols('lnc ~ lnq + lf + lnpf + C(id)', data=df).fit()
df['e'] = model.resid

# 2. Estimar varianza log-lineal por grupo (Harvey)
df['ln_e2'] = np.log(df['e']**2)

# Regresión de ln(e^2) ~ efectos fijos de grupo
var_model = smf.ols('ln_e2 ~ C(id)', data=df).fit()
df['ln_v'] = var_model.fittedvalues

# 3. Obtener pesos FGLS: v = exp(ln_v)
df['v'] = np.exp(df['ln_v'])

# 4. Estimación FGLS con pesos = 1/v
fgls_model = smf.wls('lnc ~ lnq + lf + lnpf + C(id)', data=df, weights=1/df['v']).fit()

print(fgls_model.summary())

                            WLS Regression Results                            
Dep. Variable:                    lnc   R-squared:                       0.998
Model:                            WLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     4531.
Date:                Sun, 25 May 2025   Prob (F-statistic):          5.09e-104
Time:                        17:19:02   Log-Likelihood:                 133.03
No. Observations:                  90   AIC:                            -248.1
Df Residuals:                      81   BIC:                            -225.6
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      9.8414      0.177     55.649      0.0

In [18]:
# Ajuste de intercepto (opcional, informativo)
intercepto_crudo = var_model.params['Intercept']
intercepto_corregido = intercepto_crudo + 1.2704
print(f"Intercepto corregido de Harvey: {intercepto_corregido:.4f}")

Intercepto corregido de Harvey: -6.2137


In [22]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pyreadstat as st
import numpy as np
import matplotlib.pyplot as plt


path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 1T\[MT09] Econometría de Datos de Panel\Clases prácticas\PS 2-20250525\data\cornwell.dta"

df, meta = st.read_dta(path)
df.head(1)

,county,year,crmrte,prbarr,prbconv,prbpris,avgsen,polpc,density,taxpc,west,central,urban,pctmin80,wcon,wtuc,wtrd,wfir,wser,wmfg,wfed,wsta,wloc,mix,pctymle,d82,d83,d84,d85,d86,d87,lcrmrte,lprbarr,lprbconv,lprbpris,lavgsen,lpolpc,ldensity,ltaxpc,lwcon,lwtuc,lwtrd,lwfir,lwser,lwmfg,lwfed,lwsta,lwloc,lmix,lpctymle,lpctmin,clcrmrte,clprbarr,clprbcon,clprbpri,clavgsen,clpolpc,cltaxpc,clmix
0,1,81,0.039885,0.289696,0.402062,0.472222,5.61,0.001787,2.307159,25.69763,0,1,0,20.2187,206.480301,333.62088,182.333008,272.449188,215.73349,229.119995,409.369995,236.240005,231.470001,0.099918,0.087697,0,0,0,0,0,0,-3.221757,-1.238923,-0.911149,-0.750306,1.724551,-6.32734,0.836017,3.246399,5.330205,5.810005,5.205835,5.607452,5.374044,5.434246,6.014619,5.464848,5.44445,-2.303407,-2.43387,3.006608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630 entries, 0 to 629
Data columns (total 59 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   county    630 non-null    int64  
 1   year      630 non-null    int64  
 2   crmrte    630 non-null    float64
 3   prbarr    630 non-null    float64
 4   prbconv   630 non-null    float64
 5   prbpris   630 non-null    float64
 6   avgsen    630 non-null    float64
 7   polpc     630 non-null    float64
 8   density   630 non-null    float64
 9   taxpc     630 non-null    float64
 10  west      630 non-null    int64  
 11  central   630 non-null    int64  
 12  urban     630 non-null    int64  
 13  pctmin80  630 non-null    float64
 14  wcon      630 non-null    float64
 15  wtuc      630 non-null    float64
 16  wtrd      630 non-null    float64
 17  wfir      630 non-null    float64
 18  wser      630 non-null    float64
 19  wmfg      630 non-null    float64
 20  wfed      630 non-null    float6

In [4]:
import statsmodels.formula.api as smf

# Asegurarse que 'year' es categórica y toma 1981 como base
df['year'] = df['year'].astype('category')
df['year'] = df['year'].cat.reorder_categories(sorted(df['year'].unique()), ordered=True)
df['year'] = df['year'].cat.remove_unused_categories()

# Fórmula: incluir dummies de 'year', con base en 1981 implícita
formula = 'lcrmrte ~ lprbarr + lprbconv + lprbpris + lavgsen + lpolpc + C(year, Treatment(reference=81))'

# Ajustar modelo
model = smf.ols(formula, data=df).fit(cov_type='cluster', cov_kwds={'groups': df['county']})

# Resultados
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                lcrmrte   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.562
Method:                 Least Squares   F-statistic:                     37.19
Date:                Sun, 25 May 2025   Prob (F-statistic):           1.43e-28
Time:                        21:27:53   Log-Likelihood:                -276.51
No. Observations:                 630   AIC:                             577.0
Df Residuals:                     618   BIC:                             630.4
Df Model:                          11                                         
Covariance Type:              cluster                                         
                                             coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

In [7]:
grouped = df.groupby('county')

# Variable dependiente
df['mean_lcrmrte'] = grouped['lcrmrte'].transform('mean')

# Lista de regresores
xlist = ['lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc',
         'd82', 'd83', 'd84', 'd85', 'd86', 'd87']

# Crear medias por county para cada regresor
for var in xlist:
    df[f'mean_{var}'] = grouped[var].transform('mean')


df['within_lcrmrte'] = df['lcrmrte'] - df['mean_lcrmrte']

for var in xlist:
    df[f'within_{var}'] = df[var] - df[f'mean_{var}']


import statsmodels.api as sm

# Variables "within"
within_vars = [f'within_{var}' for var in xlist]

# Matriz de diseño sin constante
X_within = df[within_vars]
y_within = df['within_lcrmrte']

# Ajustar modelo sin intercepto
model = sm.OLS(y_within, X_within).fit()

# Mostrar resumen
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:         within_lcrmrte   R-squared (uncentered):                   0.434
Model:                            OLS   Adj. R-squared (uncentered):              0.424
Method:                 Least Squares   F-statistic:                              43.19
Date:                Sun, 25 May 2025   Prob (F-statistic):                    2.11e-69
Time:                        21:38:10   Log-Likelihood:                          405.58
No. Observations:                 630   AIC:                                     -789.2
Df Residuals:                     619   BIC:                                     -740.3
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
                      coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

In [11]:
from linearmodels.panel import PanelOLS
import pandas as pd

# Asegurar que el DataFrame tenga un índice multi-nivel: (unidad, tiempo)
# Suponiendo que tus columnas se llaman 'county' y 'year'
df = df.set_index(['county', 'year'])

# Lista de regresores
xvars = ['lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc',
         'd82', 'd83', 'd84', 'd85', 'd86', 'd87']

# Matrices de diseño
y = df['lcrmrte']
X = df[xvars]

# Modelo de efectos fijos
model = PanelOLS(y, X, entity_effects=True)
results = model.fit()
print(results.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                lcrmrte   R-squared:                        0.4342
Estimator:                   PanelOLS   R-squared (Between):              0.7929
No. Observations:                 630   R-squared (Within):               0.4342
Date:                Sun, May 25 2025   R-squared (Overall):              0.7921
Time:                        21:42:36   Log-likelihood                    405.58
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      36.911
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(11,529)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             36.911
                            

In [17]:
# No cierra R between y overall.

In [19]:
from linearmodels.panel import PanelOLS
import pandas as pd

# Asegurarse de tener el panel bien indexado
df = df.set_index(['county', 'year'])

# Fórmula del modelo
formula = 'lcrmrte ~ 1 + lprbarr + lprbconv + lprbpris + lavgsen + lpolpc + d82 + d83 + d84 + d85 + d86 + d87 + EntityEffects'

# Estimar con errores estándar agrupados por county
model = PanelOLS.from_formula(formula, data=df)
result = model.fit(cov_type='clustered', cluster_entity=True)

# Mostrar resumen
print(result.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                lcrmrte   R-squared:                        0.4342
Estimator:                   PanelOLS   R-squared (Between):              0.3746
No. Observations:                 630   R-squared (Within):               0.4342
Date:                Sun, May 25 2025   R-squared (Overall):              0.3798
Time:                        21:51:51   Log-likelihood                    405.58
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      36.911
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(11,529)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             11.601
                            

In [21]:
# No cierra a detalle standard errors, pero sí los coeficientes.

In [23]:
import pandas as pd
import statsmodels.api as sm
from linearmodels.panel import PooledOLS

# Asegúrate de que el DataFrame esté indexado por (panel, tiempo)
df = df.sort_values(['county', 'year'])  # Ajustar nombres si son distintos
df = df.set_index(['county', 'year'])

# Lista de variables
xlist = ['lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc', 'd82', 'd83', 'd84', 'd85', 'd86', 'd87']

# Diferencias primeras
df['dlcrmrte'] = df['lcrmrte'] - df['lcrmrte'].groupby('county').shift(1)
for var in xlist:
    df[f'd{var}'] = df[var] - df[var].groupby('county').shift(1)

# Eliminar filas con NA (por diferencia)
df_diff = df.dropna(subset=['dlcrmrte'] + [f'd{var}' for var in xlist])

# Definir y (sin constante) y X (ya transformadas)
y = df_diff['dlcrmrte']
X = df_diff[[f'd{var}' for var in xlist]]

# Estimación sin constante
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               dlcrmrte   R-squared:                       0.433
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     40.32
Date:                Sun, 25 May 2025   Prob (F-statistic):           6.30e-59
Time:                        21:59:22   Log-Likelihood:                 248.48
No. Observations:                 540   AIC:                            -475.0
Df Residuals:                     529   BIC:                            -427.7
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
dlprbarr      -0.3275      0.030    -10.924      0.0